# 00 - Kanton Zürich - Map Widget
### Author: Daniel Herrera-Russert
#### February 03, 2025

## 1. Querying Geothermal Probe Allowance Based on Location

To enable a seamless querying mechanism for determining the allowance status of a given location, we introduce a function that performs spatial queries on the processed geographical allowability dataset. The ultimate goal is to integrate this functionality into a UI-based application featuring an interactive map widget. 

Since standard map tile providers operate in **WGS84 coordinates**, while Swiss geospatial datasets maintain higher accuracy in **LV95 (Swiss Coordinate System)**, we will preserve all datasets in LV95 for compatibility with other Swiss geodata. However, user inputs will be provided in WGS84 and must be converted accordingly before performing spatial operations.

The following code is based on a function which takes **WGS84 coordinates as input**, converts them to LV95, and checks their intersection with the preprocessed restriction polygons. If a match is found, it returns the corresponding restriction category as a string. 

Additionaly, the depth limitation function, which is imported as a package named `depth_query`, retrieves elevation and maximum allowable borehole depth for a given coordinate in the Canton of Zürich by making an HTTP GET request to the Zürich maps API. First, the input coordinates in WGS84 format are transformed into the Swiss LV95 coordinate system using `pyproj.Transformer`, as the API requires queries in LV95. The function then constructs a URL with the converted coordinates and sends a request using the `requests` library. The response, which is in HTML format, is parsed using `BeautifulSoup` to extract relevant information. Specifically, it searches for numerical values associated with elevation (marked as "Höhe") and depth constraints (marked as "Meter ab Terrain") using regular expressions.

In [1]:
import os
import ipyleaflet
import geopandas as gpd
import numpy as np
import json
import pyproj
from shapely.geometry import Point
import ipywidgets as widgets
from scripts.depth_query import get_depth_info # Depth calculator
from scipy.spatial import cKDTree # For GeoJSON queries
import joblib # To load the xgboost model
import time
from threading import Timer

In [2]:
# For colored prints :)
class style():
  RED = '\033[31m'
  GREEN = '\033[32m'
  BLUE = '\033[34m'
  RESET = '\033[0m'

In [2]:
# Load the cleaned dataset
zh_cleaned_gdf = gpd.read_file("data/transformed/zh_combined_restrictions.geojson")

# Ensure the dataset is in LV95 (Swiss Coordinate System)
zh_cleaned_gdf = zh_cleaned_gdf.to_crs("EPSG:2056")

# Load the Zürich boundary GeoJSON
boundary_gdf = gpd.read_file("data/raw/zh_boundary.geojson")
boundary_geojson = json.loads(boundary_gdf.to_json())  # Convert to JSON format

# Load borehole dataset
geojson_path = "data/transformed/zh_geothermal_probes_with_density_elevation.geojson"
zh_geothermal_probes_gdf = gpd.read_file(geojson_path)

# Create spatial index for fast nearest neighbor search
borehole_coords = np.vstack((zh_geothermal_probes_gdf.geometry.x, zh_geothermal_probes_gdf.geometry.y)).T
borehole_tree = cKDTree(borehole_coords)

---

## 2. Drag a point, evaluate allowability and query GIS data

The first iteration of the tool concept is an interactive map widget which incorporates a draggable pin point, which updates the output information based on the underlying datasets. Based on each new position, the code performs the following:

- If the point lands outside the Canton boundaries, a warning is raised and no further steps are made.
- If the point lands within the Canton boundaries:
    -  The **elevation** above sea level is scraped from the Zürich GIS portal, and the following conditions are evaluated:
        -  If the location is `Not allowed` for geothermal surface heat probes (EWS), a warning is raised and no further steps are made.
        -  If the location is flagged as `Allowed with conditions` or `Allowed`, it is indicated as output, and the **maximum allowed probe depth** in that area is scraped from the GIS portal and included in the output.

In [9]:
# Initialize ipyleaflet map centered on Zürich
m = ipyleaflet.Map(center=(47.3769, 8.5417), zoom=10)

# Create a GeoJSON layer for the boundary
boundary_layer = ipyleaflet.GeoJSON(
    data=boundary_geojson,
    style={"color": "gray", "fillOpacity": 0.2},  # Boundary style
    name="Zürich Boundary"
)

# Add the boundary layer to the map
m.add_layer(boundary_layer)

# Create an output widget
output = widgets.Output()

# Create a draggable marker
marker = ipyleaflet.Marker(location=(47.3769, 8.5417), draggable=True)
m.add_layer(marker)

# Define function for coordinate conversion & restriction query
def check_restriction(lat, lon):
    """Takes WGS84 latitude & longitude and returns the restriction status."""
    
    # Convert user coordinates from WGS84 to LV95
    wgs84_to_lv95 = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:2056", always_xy=True)
    lv95_x, lv95_y = wgs84_to_lv95.transform(lon, lat)
    
    # Create a point in LV95
    user_point_lv95 = Point(lv95_x, lv95_y)

    # Check if the point falls within any restriction area
    matching_row = zh_cleaned_gdf[zh_cleaned_gdf.contains(user_point_lv95)]

    if not matching_row.empty:
        return matching_row.iloc[0]["restrictions"]
    else:
        return f"{style.RED}❌ Outside of Canton Limits!{style.RESET}"

# Function to handle marker drag event
def on_marker_drag(change):
    """Triggered when marker is dragged to a new location."""
    lat, lon = marker.location  # Get new marker location
    
    # Query restriction status
    restriction_value = check_restriction(lat, lon)

    # Query depth information
    elevation, depth_max = get_depth_info(lat, lon)
    
    # Display result
    with output:
        output.clear_output()
        print(f"📍 Dragged to: ({lat:.5f}, {lon:.5f}) → Status: {restriction_value}")
        print(f"🏔️ Elevation: {elevation} | 🔽 Max Depth: {depth_max}")

# Attach event listener for dragging
marker.observe(on_marker_drag, "location")

# Display map and output
display(m, output)

Map(center=[47.3769, 8.5417], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

Output()

---

## 3. Adding calculation of independent variables from dragged point

The second step involves incorporating the transformed borehole dataset, and preparing the point-specific variables for the predictive model. The following code is a proof of concept, as it is expected that some of the predictive variables will be input by the user in the final application tool. It also incorporates normalization operations which set the data to the same format used for training the predictive model. This iteration expands on the first one by adding the following features:

- For any location flagged as `Allowed` and `Allowed with conditions`, data is collected and coded the same as in the borehole dataset:
    - `elevation`: is scraped from the GIS portal.
    - `Gesamtsondenzahl`: *is assigned a default value for now*.
    - `count_100m`: for each new dragged position, the number of neighbouring probes within a 100 m radius are queried and stored.
    - `nearest_borehole_dist`: for each new dragged position, the distance to the nearest neighbouring borehole is calculated.
    - `count_100m_norm`: the new `count_100m` value is normalized according to the other data and stored.
    - `neares_borehole_dist_norm`: the new `nearest_borehole_dist` value is normalized according to the other data and stored.
    - `Sondentiefe`: *the maximum allowed is set as the probe depth for now* .
    - `bottom_elevation`: the elevation of the lowest point of the borehole is calculated by subtracting the depth value from the elevation.

The comments in cursive represent variables which are for the moment set with default values, but are planned to be interactive in a future iteration.

In [11]:
# Function to compute features
def compute_features(lat, lon):
    """Computes model input features from dragged map point."""
    
    # Convert coordinates to Swiss LV95
    wgs84_to_lv95 = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:2056", always_xy=True)
    lv95_x, lv95_y = wgs84_to_lv95.transform(lon, lat)
    user_point = Point(lv95_x, lv95_y)

    # Check if the point is in an allowed area
    restriction_status = zh_cleaned_gdf[zh_cleaned_gdf.contains(user_point)]
    if restriction_status.empty:
        return f"{style.RED}❌ Outside of Canton Limits!{style.RESET}", None
    restriction_value = restriction_status.iloc[0]["restrictions"]
    if restriction_value not in ["Allowed", "Allowed with conditions"]:
        return f"⛔ {restriction_value}", None

    # Fetch elevation and depth
    elevation, depth_max = get_depth_info(lat, lon)
    
    # Count boreholes within 100m
    neighbors_within_100m = borehole_tree.query_ball_point([lv95_x, lv95_y], 100)
    count_100m = len(neighbors_within_100m)

    # Replace NaN values with 0 for computed features
    count_100m = count_100m if not np.isnan(count_100m) else 0

    # Compute nearest borehole distance
    if neighbors_within_100m:
        distances, _ = borehole_tree.query([lv95_x, lv95_y], k=2)  # k=2 to ignore self
        nearest_borehole_dist = distances[1]  # Second element is the nearest neighbor

    # Normalize using max values in dataset
    count_100m_norm = count_100m / zh_geothermal_probes_gdf["count_100m"].max()
    nearest_borehole_dist_norm = nearest_borehole_dist / zh_geothermal_probes_gdf["nearest_borehole_dist"].max()

    # User input (or fallback to depth max)
    sondentiefe = depth_max  # Placeholder, will later be user-defined
    bottom_elevation = elevation - sondentiefe

    # Return computed values
    return restriction_value, {
        "Gesamtsondenzahl": 3,  # Placeholder user input
        "count_100m": count_100m,
        "nearest_borehole_dist": nearest_borehole_dist,
        "count_100m_norm": count_100m_norm,
        "nearest_borehole_dist_norm": nearest_borehole_dist_norm,
        "Sondentiefe (max)": sondentiefe,
        "bottom_elevation": bottom_elevation
    }

# Update map event
def on_marker_drag(change):
    lat, lon = marker.location
    restriction_status, features = compute_features(lat, lon)
    
    with output:
        output.clear_output()
        print(f"📍 Dragged to: ({lat:.5f}, {lon:.5f}) → Status: {restriction_status}")
        if features:
            print("Computed Features for Model Input:")
            for key, value in features.items():
                print(f"🔹 {key}: {value:.4f}")

# Initialize map
m = ipyleaflet.Map(center=(47.3769, 8.5417), zoom=10)
marker = ipyleaflet.Marker(location=(47.3769, 8.5417), draggable=True)
m.add_layer(marker)

# Add the boundary layer to the map
boundary_layer = ipyleaflet.GeoJSON(
    data=boundary_geojson,
    style={"color": "gray", "fillOpacity": 0.2},  # Boundary style
    name="Zürich Boundary"
)
m.add_layer(boundary_layer)

output = widgets.Output()
marker.observe(on_marker_drag, "location")

display(m, output)

Map(center=[47.3769, 8.5417], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

Output()

---

## 4. Adding predicted energy yield from model

In [12]:
# Load the trained XGBoost model
xgb_model = joblib.load("models/xgb_zh_energy_yield.pkl")
print("✅ Model Loaded Successfully")

# Function to predict energy yield
def predict_energy_yield(features):
    """Predict energy yield using the trained XGBoost model."""
    features_array = np.array(features).reshape(1, -1)  # Convert to 2D array
    prediction = xgb_model.predict(features_array)[0]  # Predict and extract single value
    return prediction

# Function to compute features
def compute_features(lat, lon):
    """Computes model input features from dragged map point."""
    
    # Convert coordinates to Swiss LV95
    wgs84_to_lv95 = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:2056", always_xy=True)
    lv95_x, lv95_y = wgs84_to_lv95.transform(lon, lat)
    user_point = Point(lv95_x, lv95_y)

    # Check if the point is in an allowed area
    restriction_status = zh_cleaned_gdf[zh_cleaned_gdf.contains(user_point)]
    if restriction_status.empty:
        return f"{style.RED}❌ Outside of Canton Limits!{style.RESET}", None
    restriction_value = restriction_status.iloc[0]["restrictions"]
    if restriction_value not in ["Allowed", "Allowed with conditions"]:
        return f"⛔ {restriction_value}", None

    # Fetch elevation and depth
    elevation, depth_max = get_depth_info(lat, lon)
    
    # Count boreholes within 100m
    neighbors_within_100m = borehole_tree.query_ball_point([lv95_x, lv95_y], 100)
    count_100m = len(neighbors_within_100m)

    # Replace NaN values with 0 for computed features
    count_100m = count_100m if not np.isnan(count_100m) else 0

    # Compute nearest borehole distance
    if neighbors_within_100m:
        distances, _ = borehole_tree.query([lv95_x, lv95_y], k=2)  # k=2 to ignore self
        nearest_borehole_dist = distances[1]  # Second element is the nearest neighbor

    # Normalize using max values in dataset
    count_100m_norm = count_100m / zh_geothermal_probes_gdf["count_100m"].max()
    nearest_borehole_dist_norm = nearest_borehole_dist / zh_geothermal_probes_gdf["nearest_borehole_dist"].max()

    # User input (or fallback to depth max)
    sondentiefe = depth_max  # Placeholder, will later be user-defined
    bottom_elevation = elevation - sondentiefe

    # Return computed values
    return restriction_value, {
        "Gesamtsondenzahl": 3,  # Placeholder user input
        "count_100m": count_100m,
        "nearest_borehole_dist": nearest_borehole_dist,
        "count_100m_norm": count_100m_norm,
        "nearest_borehole_dist_norm": nearest_borehole_dist_norm,
        "Sondentiefe (max)": sondentiefe,
        "bottom_elevation": bottom_elevation
    }

# Update map event
def on_marker_drag(change):
    lat, lon = marker.location
    restriction_status, features = compute_features(lat, lon)
    
    with output:
        output.clear_output()
        print(f"📍 Dragged to: ({lat:.5f}, {lon:.5f}) → Status: {restriction_status}")
        if features:
            print("Computed Features for Model Input:")
            for key, value in features.items():
                print(f"🔹 {key}: {value:.4f}")

# Initialize map
m = ipyleaflet.Map(center=(47.3769, 8.5417), zoom=10)
marker = ipyleaflet.Marker(location=(47.3769, 8.5417), draggable=True)
m.add_layer(marker)

# Add the boundary layer to the map
boundary_layer = ipyleaflet.GeoJSON(
    data=boundary_geojson,
    style={"color": "gray", "fillOpacity": 0.2},  # Boundary style
    name="Zürich Boundary"
)
m.add_layer(boundary_layer)

output = widgets.Output()
marker.observe(on_marker_drag, "location")

display(m, output)

✅ Model Loaded Successfully


Map(center=[47.3769, 8.5417], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

Output()